In [149]:
import pandas as pd
dataset = pd.read_csv("loan.csv")

In [150]:
nFeatures = len(dataset.columns) - 1 # -1 is for removing y column
columns = list(dataset.columns)
columnsValues = dict()
# getting all possible value per feature
# also getting all possible y outputs
for row in dataset.itertuples():
    for c,value in zip(columns,list(row[1:])):
        if columnsValues.get(c) == None:
            columnsValues[c] = [value]
        else:
            if value not in columnsValues[c]:
                columnsValues[c].append(value)        
featuresName = columns[:-1] # removing y
for s in columnsValues:
    print(s,"->",columnsValues[s])

reputation -> ['excellent', 'bad', 'good']
lenght -> [5, 3]
income -> ['high', 'modest']
y -> ['safe', 'risky']


In [151]:
class Node:
    def __init__(self):
        self.parent = None
        self.feature = None
        self.link = None
        self.childs = None
        self.y = None
        
    # get all features values from this node to the root (in order)
    def getRoute(self):
        if self.parent == None:
            if self.link == None:
                return []
            return [self.link]
        return [self.link] + self.parent.getRoute()
    
    # get all features names from this node to the root (in order)
    def getFeaturesRoute(self):
        if self.parent == None: return []
        return [self.parent.feature] + self.parent.getFeaturesRoute()

class Tree:
    def __init__(self):
        self.root = Node()

    # get a list of leaves from node (if node = None, it is from root)
    def getLeaves(self, node=None):
        if node == None: node = self.root
        if node.childs == None:
            return [node]
        return self.__getLeavesNodes(node.childs)
    
    # get a list of leaves from a list of nodes
    def __getLeavesNodes(self, nodes):
        output = []
        for node in nodes:
            output.extend(self.getLeaves(node))
        return output
    
    # prediction
    def eval(self, row, names):
        node = self.root
        while node.y == None:
            i = names.index(node.feature)
            value = row[i]
            for child in node.childs:
                if child.link == value:
                    node = child
        return node.y
    
    # nodes number, utility function
    def nodesNumber(self, node = None):
        if node == None:
            node = self.root
        if node.childs == None: return 1
        else:
            count = 1
            for c in node.childs:
                count += self.nodesNumber(c)
            return count


In [152]:
# building tree without pruning
tree = Tree()
Y = columnsValues["y"]
for level in range(len(featuresName)):
    leaves = tree.getLeaves()
    # expanding each leaf
    for l in leaves:
        # checking what features are not used to reach this leaf
        remainingFeatures = []
        routeName = l.getFeaturesRoute()
        route = l.getRoute()
        for f in featuresName:
            if f not in routeName:
                remainingFeatures.append(f)
        # getting all possible legit outputs
        temp = dict()
        for row in dataset.itertuples():
            check = True
            # checking if row actually brings me to this particular leaf
            for fea,feaVal in zip(routeName,route):
                i = featuresName.index(fea)
                if feaVal != row[1:-1][i]:
                    check = False
            if check:
                for elem,name in zip(row[1:-1],featuresName):
                    if temp.get((name,elem,row[-1])) == None:
                        temp[(name,elem,row[-1])] = 1
                    else:
                        temp[(name,elem,row[-1])] += 1
        # figuring what feature minimizes the prediction error
        minErrorFeature = None
        for f in remainingFeatures:
            totCount = 0
            best = [] # best y value (less errors) per possible feature f value
            for v in columnsValues[f]:
                maxValue = 0
                maxValueY = None
                for y in Y:
                    if (temp.get((f,v,y))) != None:
                        totCount += (temp.get((f,v,y)))
                        if temp.get((f,v,y)) > maxValue:
                            maxValue = temp.get((f,v,y))
                            maxValueY = y
                best.append((v,maxValue,maxValueY))
            good = 0
            for _, number, _ in best:
                good += number
            errors = totCount - good
            errors = errors / totCount
            if minErrorFeature == None:
                minErrorFeature = (errors, f, best)
            else:
                if errors < minErrorFeature[0]:
                   minErrorFeature = (errors, f, best)
        # adding childs with corrisponding feature and feature's value
        l.feature = minErrorFeature[1]
        # print(minErrorFeature)
        childs = []
        for value,_,y in minErrorFeature[2]:
            child = Node()
            childs.append(child)
            child.parent = l
            child.link = value
            if level+1 == len(featuresName): # if its last possible level, save y output
                child.y = y
        l.childs = childs

In [153]:
# building tree with pruning
tree2 = Tree()
Y = columnsValues["y"]
for level in range(len(featuresName)):
    leaves = tree2.getLeaves()
    # expanding each leaf
    for l in leaves:
        if l.y != None:
            continue
        # checking what features are not used to reach this leaf
        remainingFeatures = []
        routeName = l.getFeaturesRoute()
        route = l.getRoute()
        for f in featuresName:
            if f not in routeName:
                remainingFeatures.append(f)
        # getting all possible legit outputs
        temp = dict()
        for row in dataset.itertuples():
            check = True
            # checking if row actually brings me to this particular leaf
            for fea,feaVal in zip(routeName,route):
                i = featuresName.index(fea)
                if feaVal != row[1:-1][i]:
                    check = False
            if check:
                for elem,name in zip(row[1:-1],featuresName):
                    if temp.get((name,elem,row[-1])) == None:
                        temp[(name,elem,row[-1])] = 1
                    else:
                        temp[(name,elem,row[-1])] += 1
        # figuring what feature minimizes the prediction error
        minErrorFeature = None
        for f in remainingFeatures:
            totCount = 0
            best = [] # best y value (less errors) per possible feature f value
            for v in columnsValues[f]:
                c = 0
                maxValue = 0
                maxValueY = None
                for y in Y:
                    if (temp.get((f,v,y))) != None:
                        c += temp.get((f,v,y))
                        totCount += temp.get((f,v,y))
                        if temp.get((f,v,y)) > maxValue:
                            maxValue = temp.get((f,v,y))
                            maxValueY = y
                zeroErrorsColumn = (c == maxValue)
                best.append((v,maxValue,maxValueY,zeroErrorsColumn))
            good = 0
            for _, number, _, _ in best:
                good += number
            errors = totCount - good
            errors = errors / totCount
            if minErrorFeature == None:
                minErrorFeature = (errors, f, best)
            else:
                if errors < minErrorFeature[0]:
                   minErrorFeature = (errors, f, best)
        # adding childs with corrisponding feature and feature's value
        l.feature = minErrorFeature[1]
        # print(minErrorFeature)
        childs = []
        for value,_,y,zeroErrors in minErrorFeature[2]:
            child = Node()
            childs.append(child)
            child.parent = l
            child.link = value
            if level+1 == len(featuresName) or zeroErrors: # if its last possible level, save y output
                child.y = y
        l.childs = childs

In [154]:
# checking prediction error rate on the dataset
errors = 0
predictions = 0
for row in dataset.itertuples():
    predictions += 1
    prediction = tree.eval(row[1:-1], featuresName)
    if prediction != row[-1]:
        errors += 1
print("accuracy =", 1-errors/predictions)

accuracy = 0.775


In [155]:
# checking prediction error rate on the dataset
errors = 0
predictions = 0
for row in dataset.itertuples():
    predictions += 1
    prediction = tree2.eval(row[1:-1], featuresName)
    if prediction != row[-1]:
        errors += 1
print("accuracy =", 1-errors/predictions)

accuracy = 0.775


In [157]:
print(tree.nodesNumber())
print(tree2.nodesNumber())

22
16
